In [404]:
import glob
import zipfile 
import os
import xml.etree.ElementTree as ET
import collections
import csv
import random
from bs4 import BeautifulSoup

In [298]:
    paths_nicaa = ['GESTION/REFERENCE/IDWEB',
                   'GESTION/MARCHE/ANNONCE_ANTERIEUR/REFERENCE/IDWEB',
                   'DONNEES/IDENTITE/',
                   'DONNEES/OBJET/OBJET_COMPLET']

In [393]:
paths = glob.glob('2005/*/*.xml')
k = 0
for path in paths:
    file = open(path, encoding='ISO-8859-1')
    apple = file.read()
    apple = apple.replace('&apos;', "'")
    apple = apple.replace('&deg;', '°')
    apple = apple.replace('&quot;', '"')
    tree = ET.fromstring(apple)
    annonces = tree.getchildren()
    
    k+= len(annonces)

In [ ]:
def xml_parser(paths, file):
    
    for path in paths:
        for element in root.find(path):
            pairs.append([element.tag, element.text])
    return pairs

In [320]:
d = annonces[0].find('DONNEES')

In [367]:
annonces[0].find('GESTION/').text

'BWP053410400730'

In [387]:
annonces[0].find('DONNEES').getchildren()

[<Element 'INFO' at 0x7f81871a8e08>,
 <Element 'IDENT' at 0x7f81871a8b38>,
 <Element 'OBJET' at 0x7f8186f14048>,
 <Element 'CARACTERISTIQUES' at 0x7f8186e0cdb8>,
 <Element 'DUREE' at 0x7f8186e0cb38>,
 <Element 'CONDITIONS' at 0x7f8186e0ce08>,
 <Element 'PROCEDURE' at 0x7f8186f09048>,
 <Element 'DELAI' at 0x7f8184b146d8>,
 <Element 'RENSEIGNEMENTS' at 0x7f8184b14048>,
 <Element 'ENVOI_BOMP' at 0x7f8184b14138>,
 <Element 'CORRESPONDANTS' at 0x7f8184b14368>,
 <Element 'DOCUMENTS' at 0x7f8184b14228>,
 <Element 'LOTS' at 0x7f8186a00048>,
 <Element 'CLASSES' at 0x7f81869a0a98>,
 <Element 'AUTRES' at 0x7f81869a0a48>,
 <Element 'AUTRES' at 0x7f81869a0908>,
 <Element 'DESCRIPTEURS' at 0x7f81869a0778>,
 <Element 'AUTRES' at 0x7f81869a0818>]

In [415]:
soup = BeautifulSoup(apple, 'lxml')

In [420]:
soup.find_all('DONNEES')

[]